pretty_midi
直接生成guitar tab
Guitar Fret

In [ ]:
class Guitar():
    def __init__(self,tuning=[40, 45, 50, 55, 59, 64], fretnum=24):
        self.tuning= tuning
        self.fretnum=fretnum
        def genfboard():
            boarddic={}
            stri=6
            for i in self.tuning:
                boarddic[stri]=[]
                for j in range(self.fretnum+1):
                    boarddic[stri].append(i+j)
                stri-=1
            return boarddic
        self.fboard=genfboard()
        self.chords = {
            "C": [  # 再细化，只包含可替换的和弦，根据chords with color来
                    # CAGED 根据规则
                {6: 0, 5: 3, 4: 2, 3: 0, 2: 1, 1: 0},#Triad
                {6: 0, 5: 3, 4: 0, 3: 0, 2: 3, 1: 3},#sus2
                {6: 0, 5: 3, 4: 3, 3: 0, 2: 1, 1: 3},#sus4
                {6: 0, 5: 3, 4: 2, 3: 0, 2: 3, 1: 3},#add9
                {6: 0, 5: 3, 4: 2, 3: 2, 2: 1, 1: 0},#6
                {6: 0, 5: 3, 4: 2, 3: 0, 2: 0, 1: 0},#7
                {6: 0, 5: 3, 4: 2, 3: 4, 2: 3, 1: 0}#9
            ],
            "D": [
                {6: 0, 5: 0, 4: 0, 3: 2, 2: 3, 1: 1},#Triad
                {6: 0, 5: 0, 4: 0, 3: 2, 2: 3, 1: 0},#sus2
                {6: 0, 5: 0, 4: 0, 3: 2, 2: 3, 1: 3},#sus4
                {6: 0, 5: 0, 4: 3, 3: 2, 2: 3, 1: 0},#add9
                {6: 0, 5: 0, 4: 0, 3: 2, 2: 0, 1: 1},#6
                {6: 0, 5: 0, 4: 0, 3: 2, 2: 1, 1: 1},#7
                {6: 0, 5: 5, 4: 3, 3: 5, 2: 5, 1: 0}#9
            ],
            "E": [
                {6: 0, 5: 2, 4: 2, 3: 0, 2: 0, 1: 0},#Triad
                {6: 0, 5: 2, 4: 2, 3: 2, 2: 0, 1: 0},#sus4
                {6: 0, 5: 2, 4: 2, 3: 0, 2: 3, 1: 0}#7
            ],
            "F": [
                {6: 1, 5: 3, 4: 3, 3: 2, 2: 1, 1: 1},#Triad
                {6: 0, 5: 3, 4: 3, 3: 0, 2: 1, 1: 1},#sus2
                {6: 0, 5: 0, 4: 3, 3: 2, 2: 1, 1: 3},#add9
                {6: 0, 5: 0, 4: 3, 3: 2, 2: 3, 1: 1},#6
                {6: 0, 5: 0, 4: 3, 3: 2, 2: 1, 1: 0},#7
                {6: 0, 5: 8, 4: 7, 3: 9, 2: 8, 1: 0}#9

            ],
            "G": [
                {6: 3, 5: 2, 4: 0, 3: 0, 2: 0, 1: 3},#Triad
                {6: 3, 5: 0, 4: 0, 3: 2, 2: 3, 1: 3},#sus2
                {6: 3, 5: 3, 4: 0, 3: 0, 2: 1, 1: 3},#sus4
                {6: 3, 5: 0, 4: 0, 3: 2, 2: 0, 1: 3},#6
                {6: 3, 5: 2, 4: 3, 3: 0, 2: 0, 1: 3}#7
            ],
            "A": [
                {6: 0, 5: 0, 4: 2, 3: 2, 2: 2, 1: 0}
            ],
            "B": [
                {6: 0, 5: 2, 4: 4, 3: 4, 2: 4, 1: 2}
            ],
        }

    def get_chord_midi(self, chord_fingering):

        midi_notes = []
        for string, fret in chord_fingering.items():
            midi_note = self.fboard[string][fret]
            midi_notes.append(midi_note)

        return midi_notes

class Hand():
    def __init__(self):
        self.f1=None
        self.f2=None
        self.f3=None
        self.f4=None

In [64]:
def find_best_fretboard_match(g, melody, width=4):

    max_overlap = 0
    best_section = None

    for start_fret in range(g.fretnum - width + 1):
        notes_in_range = set()
        for string, frets in g.fboard.items():
            notes_in_range.update(frets[start_fret : start_fret + width + 1])
        overlap = len(set(melody) & notes_in_range)

        if overlap > max_overlap:
            max_overlap = overlap
            best_section = (start_fret, start_fret + width)

    return best_section, max_overlap

In [65]:
def find_best_chord_match(g, m):
    max_overlap = 0
    best_chord = None

    cname=m[0]
    melody=m[1]
    if cname not in g.chords:
        raise ValueError(f"Chord '{cname}' not found in Guitar instance.")
    
    for fingering in g.chords[cname]:
        chord=g.get_chord_midi(fingering)
        overlap = len(set(melody) & set(chord))

        if overlap > max_overlap:
            max_overlap = overlap
            best_chord = (fingering,chord)

    return best_chord, max_overlap

In [ ]:
g1=Guitar()
test_melody = [40, 75, 81, 47, 80, 55]
test_melody2 = ['C', [40, 48, 52, 55, 62, 64, 80, 64]]
width = 4 

best_section, overlap = find_best_fretboard_match(g1, test_melody, width)

print(f"Best section: frets {best_section}")    
print(f"Overlap: {overlap/len(test_melody)}")

best_section, overlap = find_best_fretboard_match(g1, test_melody2[1], width)

print(f"Best Position: frets {best_section}")    
print(f"Overlap: {overlap/len(test_melody2[1])}")

Best section: frets (13, 17)
Overlap: 0.6666666666666666
Best section: frets (0, 4)
Overlap: 0.75


In [67]:
best_section, overlap = find_best_chord_match(g1, ['C',test_melody])

print(f"Best chord: {best_section}")   
print(f"Overlap: {overlap/len(test_melody)}")

best_section, overlap = find_best_chord_match(g1, test_melody2)

print(f"Best chord: {best_section}")    
print(f"Overlap: {overlap/len(test_melody2[1])}")

Best chord: ({6: 0, 5: 3, 4: 2, 3: 0, 2: 1, 1: 0}, [40, 48, 52, 55, 60, 64])
Overlap: 0.3333333333333333
Best chord: ({6: 0, 5: 3, 4: 2, 3: 0, 2: 3, 1: 0}, [40, 48, 52, 55, 62, 64])
Overlap: 0.75


In [68]:
print(g1.fboard[1])
midi_notes = g1.get_chord_midi(g1.chords['C'][0])
print(f"MIDI notes for chord: {midi_notes}")

[64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88]
MIDI notes for chord: [40, 48, 52, 55, 60, 64]
